# Import for NumPy, TensorFlow, PyTorch, Jax

In [ ]:
from phi.flow import *
from phi.tf.flow import *
from phi.torch.flow import *
from phi.jax.flow import *

# Construct `StaggeredGrid` from NumPy Arrays

Given matching arrays `vx` and `vy`, we can construct a `StaggeredGrid`.
Note that the shapes of the arrays must match the extrapolation!

In [ ]:
vx = math.tensor(np.zeros([33, 32]), spatial('x,y'))
vy = math.tensor(np.zeros([32, 33]), spatial('x,y'))
StaggeredGrid(math.stack([vx, vy], channel('vector')), extrapolation.BOUNDARY)

vx = math.tensor(np.zeros([32, 32]), spatial('x,y'))
vy = math.tensor(np.zeros([32, 32]), spatial('x,y'))
StaggeredGrid(math.stack([vx, vy], channel('vector')), extrapolation.PERIODIC)

vx = math.tensor(np.zeros([31, 32]), spatial('x,y'))
vy = math.tensor(np.zeros([32, 31]), spatial('x,y'))
StaggeredGrid(math.stack([vx, vy], channel('vector')), 0)